In [ ]:
# tokenizer를 불러오는 코드. 다른것도 있으니 한번에 정리 필요할지도?
tokenizer = tiktoken.get_encoding("gpt2")

# 사람이 읽는 텍스트 문자열을 모델이 이해하는 토큰 ID 텐서로 변환
# 파라미터로 text만 입력하는 경우도 있음, <|endoftext|> 이 특수 토큰
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

# encoding한거 다시 decoding하는 코드
strings = tokenizer.decode(integers)
############################################################################
# LLM은 input x와 label y, x를 통해 그 다음에 올 토큰을 추론하는거라 1씩 shift 된다.
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
############################################################################
# 데이터를 GPU(또는 설정된 device)로 이동
# 모델과 데이터 등 계산에 필요한 모든 것들이 같은 device로 assign되어야 함.
input = input.to(device)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
############################################################################
data_iter = iter(dataloader)
# dataloader를 iterator로 변환해준다. 

# next를 통해 다음 iterator를 불러올 수 있다. 
first_batch = next(data_iter)
# next를 반복하면 계속 다음 batch를 불러올 수 있음
first_iter = next(iter(dataloader)) # 이렇게 한번에 쓰는것도 가능하고
inputs, targets = next(data_iter) # 이렇게 해줄수도 있다. 
############################################################################

In [ ]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        # ...
        for i in range(0, len(token_ids) - max_length, stride):
            # 입력 청크: 현재 위치(i)부터 max_length만큼 가져온다. 인덱스 잘 볼것.
            input_chunk = token_ids[i : i + max_length]
                
            # 타겟 청크: 입력보다 1칸 뒤의 위치(i+1)부터 가져온다. llm이니까
            # GPT는 '다음 단어'를 맞추는 모델이므로, 정답은 입력보다 한 칸씩 뒤로 밀려있어야 한다.
            target_chunk = token_ids[i + 1 : i + max_length + 1]

            # 추출한 데이터를 텐서(Tensor)로 변환하여 리스트에 저장합니다.
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    # 토크나이저 초기화. 외우기
    tokenizer = tiktoken.get_encoding("gpt2")
    # ...
    
    return dataloader    
    

In [ ]:
def random_split(df, train_frac, validation_frac):
    """
    데이터셋을 학습(Train), 검증(Validation), 테스트(Test) 셋으로 분할하는 함수
    """
    # 전체 데이터 섞기
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)

    # 분할 지점(인덱스) 계산
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)

    # 데이터 분할
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]

    return train_df, validation_df, test_df

In [ ]:
# 임베딩 레이어 생성하는 코드
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

# 임베딩 레이어 적용 - 나올까? 싶지만 어렵진 않으니 외워두자.
# input_ids 텐서를 전달하면 embedding_layer의 각 인덱스 weight가 저장된 임베딩이 생성됨.
input_ids = torch.tensor([2, 3, 5, 1])
token_embeddings = embedding_layer(input_ids) # torch.Size([batch_size, seq_length, output_dims])

context_length = max_length
# 포지셔널 임베딩 레이어 생성
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
# 포지셔널 임베딩 적용
pos_embeddings = pos_embedding_layer(torch.arange(max_length)) # torch.Size([seq_length, output_dims])
# pos_embedding은 batch마다 따로 있는게 아니라는 점을 명심하자. 

input_embeddings = token_embeddings + pos_embeddings
# torch.Size([batch_size, seq_length, output_dims])

In [ ]:
qwen = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
# tokenizer나 model 모두 from_pretrained를 통해 받아올 수 있는듯.

In [ ]:
###########################
# 모델 저장 및 로드 테스트
###########################
torch.save(model.state_dict(), "outputs/model.pth")

# 디바이스 설정 (모델 로드 전에 정의 필요)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 다시 불러오기 테스트
model = GPTModel(GPT_CONFIG_124M) # 1. 빈 모델 생성 (현재 CPU에 있음)
model.load_state_dict(torch.load("outputs/model.pth", weights_only=True)) # 2. 가중치 로드
model.to(device) # <--- [핵심 수정] 3. 모델을 GPU로 이동시켜야 함

In [ ]:
# ==========================================
# 3. 결과 해석 (확률 변환 및 확인)
# ==========================================
# 3-1. 로짓(점수) -> 확률(Probability) 변환
# 로짓은 -무한대 ~ +무한대 범위의 숫자이므로, Softmax를 써서 0~1 사이 확률로 바꾼다
# dim=-1: 가장 마지막 차원(단어장 50257개)에 대해 확률의 합이 1이 되게 만듦
probas = torch.softmax(logits, dim=-1) 

# 3-2. 가장 높은 확률을 가진 단어 찾기 (예측 결과)
# argmax: 확률이 가장 높은 인덱스(단어 ID)를 반환
token_ids = torch.argmax(probas, dim=-1, keepdim=True)

############################################################################
# 인덱스 0에 차원을 하나 더해줌으로써 batch 차원 추가 효과를 얻을 수 있음.
encoded_tensor = torch.tensor(encoded).unsqueeze(0) 

# 디코딩 하면서 squeeze로 불필요해진 batch 차원 축소(제거) 해줌.
ecoded_text = tokenizer.decode(out.squeeze(0).tolist())


In [ ]:
# ==========================================
# 5. PyTorch 함수로 손실(Loss) 계산하기 (권장 방식)
# ==========================================

print("-" * 30)
print("로짓 크기 (변경 전):", logits.shape)   # (2, 3, 50257)
print("타깃 크기 (변경 전):", targets.shape)  # (2, 3)

# 5-1. 평탄화 (Flattening)
# CrossEntropyLoss 함수는 입력을 (N, Class) 형태로 받기를 선호합니다.
# 즉, "어떤 문장의 몇 번째 단어인지"는 중요하지 않고, "총 몇 문제를 풀었나"로 형태를 바꿉니다.

# (배치 2 * 길이 3, 단어장 50257) -> (6, 50257) : 총 6개의 단어 예측 문제로 변환
logits_flat = logits.flatten(0, 1)

# (배치 2 * 길이 3) -> (6) : 정답지도 일렬로 6개 나열
targets_flat = targets.flatten()

print("펼친 로짓:", logits_flat.shape)   # torch.Size([6, 50257])
print("펼친 타깃:", targets_flat.shape)  # torch.Size([6])

# 5-2. Cross Entropy Loss 계산
# 주의: nn.functional.cross_entropy는 입력으로 '확률(probas)'이 아니라 '로짓(logits)'을 받습니다.
# 함수 내부적으로 Softmax -> Log -> NLLLoss 과정을 모두 수행하기 때문입니다.
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)

print("함수로 계산한 손실값(Loss):", loss) 
# 이 값은 위에서 수동으로 계산한 'neg_avg_log_probas'와 거의 같아야 합니다.